<a href="https://colab.research.google.com/github/Daniel-Benson-Poe/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/LS_DS_431_RNN_and_LSTM_Assignment_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [2]:
import requests
import pandas as pd

In [3]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [4]:
#Shakespeare Data Parsed by Play
df_toc.head()

,title,start,end,text
0,ALL’S WELL THAT ENDS WELL,2777,7738,ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nConte...
1,THE TRAGEDY OF ANTONY AND CLEOPATRA,7739,11840,THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\nDRA...
2,AS YOU LIKE IT,11841,14631,AS YOU LIKE IT\r\n\r\nDRAMATIS PERSONAE.\r\n\r...
3,THE COMEDY OF ERRORS,14632,17832,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
4,THE TRAGEDY OF CORIOLANUS,17833,27806,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...


In [5]:
df_toc['title'].value_counts()

KING HENRY THE EIGHTH                       1
THE MERCHANT OF VENICE                      1
THE MERRY WIVES OF WINDSOR                  1
MEASURE FOR MEASURE                         1
THE TWO NOBLE KINSMEN                       1
LOVE’S LABOUR’S LOST                        1
KING RICHARD THE SECOND                     1
THE PHOENIX AND THE TURTLE                  1
A MIDSUMMER NIGHT’S DREAM                   1
KING RICHARD THE THIRD                      1
THE SECOND PART OF KING HENRY THE FOURTH    1
VENUS AND ADONIS                            1
THE TRAGEDY OF JULIUS CAESAR                1
THE SECOND PART OF KING HENRY THE SIXTH     1
PERICLES, PRINCE OF TYRE                    1
THE TAMING OF THE SHREW                     1
KING JOHN                                   1
THE TEMPEST                                 1
THE TRAGEDY OF ROMEO AND JULIET             1
THE LIFE OF TIMON OF ATHENS                 1
THE THIRD PART OF KING HENRY THE SIXTH      1
A LOVER’S COMPLAINT               

In [6]:
len(df_toc['text'][0])

139578

In [7]:
df_toc.shape

(43, 4)

In [8]:
sample_df_toc = df_toc[df_toc.index == 0]
sample_df_toc.shape

(1, 4)

In [9]:
sample_df_toc.head()

,title,start,end,text
0,ALL’S WELL THAT ENDS WELL,2777,7738,ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nConte...


In [10]:
# Let's do some imports
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

In [11]:
# Add all text into on single, gigantic text line
text = " ".join(sample_df_toc['text'])

# Create a list of unique characters found in the text
chars = list(set(text))
print(len(chars))

# Create lookup Tables
char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

72


In [12]:
# Create sequence data
maxlen = 150
step = 1

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 150 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):

  sequences.append(encoded[i : i + maxlen])
  next_char.append(encoded[i + maxlen])

print(f'sequences: {len(sequences)}')

sequences: 139428


In [13]:
sequences[0]

[59,
 0,
 0,
 36,
 54,
 64,
 10,
 55,
 0,
 0,
 64,
 50,
 56,
 59,
 50,
 64,
 55,
 16,
 70,
 54,
 64,
 10,
 55,
 0,
 0,
 44,
 11,
 44,
 11,
 44,
 11,
 44,
 11,
 60,
 12,
 47,
 28,
 2,
 47,
 28,
 27,
 44,
 11,
 44,
 11,
 59,
 60,
 50,
 64,
 45,
 44,
 11,
 54,
 49,
 2,
 47,
 2,
 64,
 45,
 14,
 64,
 71,
 12,
 27,
 27,
 68,
 17,
 17,
 12,
 47,
 14,
 64,
 59,
 64,
 20,
 12,
 12,
 65,
 64,
 68,
 47,
 64,
 28,
 9,
 2,
 64,
 60,
 12,
 23,
 47,
 28,
 2,
 27,
 27,
 36,
 27,
 64,
 37,
 48,
 17,
 48,
 49,
 2,
 14,
 44,
 11,
 54,
 49,
 2,
 47,
 2,
 64,
 45,
 45,
 14,
 64,
 40,
 48,
 20,
 68,
 27,
 14,
 64,
 59,
 64,
 20,
 12,
 12,
 65,
 64,
 68,
 47,
 64,
 28,
 9,
 2,
 64,
 51,
 68,
 47,
 3,
 36,
 27,
 64,
 37,
 48,
 17,
 48,
 49,
 2]

In [14]:
print(len(sequences[0]))
print(len(sequences))

150
139428


In [15]:
# Create our x and y by impletting zeros until we have 150 characters

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
  for t, char in enumerate(sequence):
    x[i, t, char] = 1
  
  y[i, next_char[i]] = 1

In [16]:
x.shape, y.shape

((139428, 150, 72), (139428, 72))

In [17]:
# Build the model

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               102912    
_________________________________________________________________
dense (Dense)                (None, 72)                9288      
Total params: 112,200
Trainable params: 112,200
Non-trainable params: 0
_________________________________________________________________


In [19]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [20]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [21]:
# Fit the model

model.fit(x, y,
          batch_size=500,
          validation_split=0.2,
          epochs=100,
          callbacks=[print_callback])

Epoch 1/100
223/224 [============================>.] - ETA: 0s - loss: 3.2385
----- Generating text after Epoch: 0
----- Generating with seed: "Nay, put me to’t, I warrant you.

COUNTESS.
You were lately whipp’d, sir, as I think.

CLOWN.
O Lord, sir! Spare not me.

COUNTESS.
Do you cr"
Nay, put me to’t, I warrant you.

COUNTESS.
You were lately whipp’d, sir, as I think.

CLOWN.
O Lord, sir! Spare not me.

COUNTESS.

OoS ne whl wtieecmt r agt on ad e ys c, n 
iter 
a se t wo i ha onko s e w e e s w e tna arrdsmd uocsd tanttn
nkyA tk I ity , t arkonn hednee nshdtr ow rtuuiloa
,.
Iyt
yd t ttaottu fowtpd mdse wtpea e fa ae
dSa
224/224 [==============================] - 34s 152ms/step - loss: 3.2383 - val_loss: 2.8704
Epoch 2/100
223/224 [============================>.] - ETA: 0s - loss: 2.5912
----- Generating text after Epoch: 1
----- Generating with seed: "a case,
That we’ll forsake ourselves. Give me that ring.

BERTRAM.
I’ll lend it thee, my dear, but have no power
To give it from me.


While the creation of our model and its text generation capabilities can be considered a success, unfortunately every time I attempt to increase the number of plays above 1 that I include in my model the notebook crashes.

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN